# Autonomous Planning Agent (Orchestration Layer)

# Libraries Import

In [ ]:
import json
from openai import OpenAI
from dotenv import load_dotenv
from Agents_Folder.Scraping_Agent import ScannerAgent
import chromadb
import logging
load_dotenv(override=True)
openai = OpenAI()
MODEL = "gpt-5.2"

SyntaxError: invalid syntax (2634931350.py, line 4)

In [2]:
test_results = ScannerAgent().test_scan()
test_results

DealSelection(deals=[Deal(product_description="The Hisense R6 Series 55R6030N is a 55-inch 4K UHD Roku Smart TV that offers stunning picture quality with 3840x2160 resolution. It features Dolby Vision HDR and HDR10 compatibility, ensuring a vibrant and dynamic viewing experience. The TV runs on Roku's operating system, allowing easy access to streaming services and voice control compatibility with Google Assistant and Alexa. With three HDMI ports available, connecting multiple devices is simple and efficient.", price=178.0, url='https://www.dealnews.com/products/Hisense/Hisense-R6-Series-55-R6030-N-55-4-K-UHD-Roku-Smart-TV/484824.html?iref=rss-c142'), Deal(product_description='The Poly Studio P21 is a 21.5-inch LED personal meeting display designed specifically for remote work and video conferencing. With a native resolution of 1080p, it provides crystal-clear video quality, featuring a privacy shutter and stereo speakers. This display includes a 1080p webcam with manual pan, tilt, and

# Defining 3 tools that represents the tasks that the planning agent needs to orchestrate

In [3]:
def scan_the_internet_for_bargains() -> str:
    # Scans the web for deals and returns a curated shortlist as a JSON string.
    """Search the web for attractive deals and return a curated shortlist of the most promising ones."""
    
    # Log that this is a mock implementation using fixed (hardcoded) results.
    print("Placeholder implementation: fake web scan performed; returning a predefined set of deals.")
    
    # Return the (hardcoded) deal results serialized to JSON.
    return test_results.model_dump_json()


In [ ]:
def estimate_true_value_of_product(description: str) -> str:
    # Estimate a product’s “true” value from its text description and return the estimate as a string.
    # (It does not run a real model and always returns $300.)
    """
    Estimate the fair-market value of a product from its textual description.
    """
    # Log that this is a mock implementation and show a short preview of the description.
    print(f"Placeholder value estimator for: {description[:20]}... (mock output: $300)")

    # Return the mock estimate in a clear, user-facing format.
    return f"Estimated fair value for the product is $300. Description: {description}"


In [10]:
# This function sends a notification to the user about a bargain. It takes description, deal price, estimated true value, and URL as inputs.
def notify_user_of_bargain(description: str, deal_price: float, estimated_true_value: float, url: str) -> str:
    # Notify the user when a deal looks attractive by comparing the deal price vs. an estimated fair value.
    # (It only prints a message and returns a success string.)
    """
    Notify the user about a high-value bargain using the product description, deal price, estimated fair value, and URL.
    """
    # Log what would be sent in a real notification system.
    print(
        f"Placeholder notification: '{description[:60]}...' | deal=${deal_price:.2f} | "
        f"estimated=${estimated_true_value:.2f} | url={url}"
    )

    # Return a mock success response.
    return "Notification (OK) sent (This is a mock test)."


# Test these tools

In [ ]:
notify_user_of_bargain("a new Apple airpods", 100, 300, "https://www.apple.com/tr/airpods/")

Placeholder notification: 'a new Apple airpods...' | deal=$100.00 | estimated=$300.00 | url=https://www.apple.com/iphone


'Notification (OK) sent (This is a mock test).'

# Tool Definitions (JSON Schema)

In [ ]:
# Tool schema for a "scan" function the LLM can call
scan_function = {
    "name": "scan_the_internet_for_bargains",  # the function/tool name the model must use when calling it
    "description": "Fetch a curated set of top deals found online, including the listed price for each item.",  # shown to the model so it knows when to use this tool

    "parameters": {                             # JSON Schema describing the tool's input arguments
        "type": "object",                       # inputs must be passed as a JSON object
        "properties": {},                       # no input fields are allowed (this tool takes no arguments)
        "required": [],                         # nothing is required (because there are no parameters)
        "additionalProperties": False           # disallow extra keys (keeps calls strict/valid)
    }
}

# Tool schema for an "estimate" function the LLM can call
estimate_function = {
    "name": "estimate_true_value",              # tool name used for model tool-calls
    "description": "Estimate the fair-market value of an item based on its text description.",  # guides the model on intent

    "parameters": {                             # JSON Schema for inputs
        "type": "object",                       # inputs must be a JSON object
        "properties": {                         # allowed fields inside that object
            "description": {                    # field name expected by Python function
                "type": "string",               # value must be a string
                "description": "A text description of the item whose value should be estimated."  # helps the model fill it correctly
            },
        },
        "required": ["description"],            # the model MUST provide "description"
        "additionalProperties": False           # forbid any other unexpected fields
    }
}

# Tool schema for a "notify" function the LLM can call
notify_function = {
    "name": "notify_user_of_deal",              # tool name used for model tool-calls
    "description": "Send a push alert for the single best deal identified; call this at most once.",  # instructs the model to use it only once

    "parameters": {                             # JSON Schema for inputs
        "type": "object",                       # inputs must be a JSON object
        "properties": {                         # allowed fields
            "description": {                    # product text/summary
                "type": "string",
                "description": "The product description extracted from the deal source."
            },
            "deal_price": {                     # price in the deal
                "type": "number",
                "description": "The deal’s advertised price."
            },
            "estimated_true_value": {           # model's estimate of fair value
                "type": "number",
                "description": "Your estimated fair value for the item."
            },
            "url": {                            # deal link
                "type": "string",
                "description": "The link to the deal page."
            }
        },
        "required": ["description", "deal_price", "estimated_true_value", "url"],  # all must be supplied
        "additionalProperties": False           # forbid extra keys beyond these four
    }
}


In [13]:
tools = [{"type": "function", "function": scan_function},
 {"type": "function", "function": estimate_function},
 {"type": "function", "function": notify_function}
 ]

In [14]:
tools

[{'type': 'function',
  'function': {'name': 'scan_the_internet_for_bargains',
   'description': 'Fetch a curated set of top deals found online, including the listed price for each item.',
   'parameters': {'type': 'object',
    'properties': {},
    'required': [],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'estimate_true_value',
   'description': 'Estimate the fair-market value of an item based on its text description.',
   'parameters': {'type': 'object',
    'properties': {'description': {'type': 'string',
      'description': 'A text description of the item whose value should be estimated.'}},
    'required': ['description'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'notify_user_of_deal',
   'description': 'Send a push alert for the single best deal identified; call this at most once.',
   'parameters': {'type': 'object',
    'properties': {'description': {'type': 'string',
      'description': 'The 

# Tool Call Handler (Execute LLM-Requested Tools)

In [ ]:
def manage_LLM_tool_response(message):
    # Execute the tools requested in an LLM message and return tool-response messages.

    results = []  # store tool outputs formatted for the chat API

    for tool_call in message.tool_calls:  # iterate over each tool call the model requested
        tool_name = tool_call.function.name  # the name of the function/tool to invoke
        arguments = json.loads(tool_call.function.arguments)  # parse JSON string -> Python dict

        tool = globals().get(tool_name)  # look up the actual Python function by name (in global scope)
        result = tool(**arguments) if tool else {}  # call the function with kwargs if found; else empty result

        # Package the result in the "tool" message format and link it back to this tool_call via tool_call_id
        results.append({
            "role": "tool",
            "content": json.dumps(result),     # convert Python result -> JSON string for the API
            "tool_call_id": tool_call.id       # ensures the API matches this output to the correct tool call
        })

    return results  # list of tool-result messages


# System and User Prompts

In [17]:
system_prompt = (
    "You are an autonomous deal-finding agent. Use the available tools to identify bargain products, "
    "estimate fair value, and notify the user about the single best opportunity."
)

user_prompt = """
Task:
1) Call scan_the_internet_for_bargains to retrieve a list of deals.
2) For each deal, call estimate_true_value using the deal's description.
3) Compute the value gap for each deal: (estimated_true_value - deal_price).
4) Select exactly ONE deal with the largest positive value gap. Prefer deals with a clear description and a reliable price.
5) Call notify_user_of_deal exactly once using: description, deal_price, estimated_true_value, and url.
6) After the notification tool call succeeds, respond with exactly: OK

Rules:
- Use tools when needed; do not invent deals or prices.
- If no deal has a positive value gap, do not notify; just respond: OK
"""
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
]


In [19]:
print(messages)

[{'role': 'system', 'content': 'You are an autonomous deal-finding agent. Use the available tools to identify bargain products, estimate fair value, and notify the user about the single best opportunity.'}, {'role': 'user', 'content': "\nTask:\n1) Call scan_the_internet_for_bargains to retrieve a list of deals.\n2) For each deal, call estimate_true_value using the deal's description.\n3) Compute the value gap for each deal: (estimated_true_value - deal_price).\n4) Select exactly ONE deal with the largest positive value gap. Prefer deals with a clear description and a reliable price.\n5) Call notify_user_of_deal exactly once using: description, deal_price, estimated_true_value, and url.\n6) After the notification tool call succeeds, respond with exactly: OK\n\nRules:\n- Use tools when needed; do not invent deals or prices.\n- If no deal has a positive value gap, do not notify; just respond: OK\n"}]


# Autonomous planning agent loop example

In [ ]:
done = False
while not done:
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    if response.choices[0].finish_reason=="tool_calls": # model wants to use a tool
        message = response.choices[0].message # get the LLM's message requesting tool calls
        results = handle_tool_call(message) # execute the tool calls and get tool-response messages
        messages.append(message) # add LLM's tool-call message to the conversation
        messages.extend(results) # add the tool-response messages to the conversation
    else: # model is done
        done = True
response.choices[0].message.content # final output from the agent

Placeholder implementation: fake web scan performed; returning a predefined set of deals.
Placeholder value estimator for: The Hisense R6 Serie... (mock output: $300)
Placeholder value estimator for: The Poly Studio P21 ... (mock output: $300)
Placeholder value estimator for: The Lenovo IdeaPad S... (mock output: $300)
Placeholder value estimator for: The Dell G15 gaming ... (mock output: $300)
Placeholder notification: 'The Poly Studio P21 is a 21.5-inch LED personal meeting disp...' | deal=$30.00 | estimated=$300.00 | url=https://www.dealnews.com/products/Poly-Studio-P21-21-5-1080-p-LED-Personal-Meeting-Display/378335.html?iref=rss-c39


'OK'

# Real Orchestration Implementation

# Logging Information

In [23]:
root_info = logging.getLogger()
root_info.setLevel(logging.INFO)

In [26]:
DataBase = "products_vectorstore"
client = chromadb.PersistentClient(path=DataBase)
collection_products = client.get_collection(name="products")

In [28]:
from agents.autonomous_planning_agent import AutonomousPlanningAgent
agent = AutonomousPlanningAgent(collection_products)

INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is initializing
INFO:root:[Scanner Agent] Scanner Agent is ready
INFO:root:[Ensemble Agent] Initializing Ensemble Agent
INFO:root:[Specialist Agent] Specialist Agent is initializing - connecting to modal
INFO:root:[OpenAI GPT 5.2 Frontier Agent] Starting OpenAI GPT 5.2 Frontier Agent
INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent is setting up with OpenAI
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent is ready
INFO:root:[Ensemble Agent] Ensemble Agent is ready
INFO:root:[Messaging Agent] Messaging Agent is initializing
INFO:root:[Messaging Agent] Messaging Agent has initialized Pushover and GPT-5-MINI
INFO:root:[Autonomous 

In [29]:
agent.plan()

INFO:root:[Autonomous Planning Agent] Autonomous Planning Agent is kicking off a run
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is calling scanner
INFO:root:[Scanner Agent] Scanner Agent is about to fetch deals from RSS feed
INFO:root:[Scanner Agent] Scanner Agent received 50 deals not already scraped
INFO:root:[Scanner Agent] Scanner Agent is calling OpenAI using Structured Outputs
INFO:root:[Scanner Agent] Scanner Agent received 10 selected deals with price>0 from OpenAI
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value via Ensemble Agent
INFO:root:[Ensemble Agent] Running Ensemble Agent - preprocessing text
14:13:41 - LiteLLM:INFO: utils.py:3421 - 
LiteLLM completion() model= openai/gpt-oss-20b; provider = groq
INFO:LiteLLM:
LiteLLM completion() model= openai/gpt-oss-20b; provider = groq
14:13:41 - LiteLLM:INFO: utils.py:1302 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent has found similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent is about to call gpt-5.2 with context including 10 similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent completed - predicting $749.00
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $709.10
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value via Ensemble Agent
INFO:root:[Ensemble Agent] Running Ensemble Agent - preprocessing text
14:14:27 - LiteLLM:INFO: utils.py:3421 - 
LiteLLM completion() model= openai/gpt-oss-20b; provider = groq
INFO:LiteLLM:
LiteLLM completion() model= openai/gpt-oss-20b; provider = groq
14:14:28 - LiteLLM:INFO: utils.py:1302 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Ensemble Agent] Pre-processed text using groq/openai/gpt-oss-20b
I

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent has found similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent is about to call gpt-5.2 with context including 10 similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent completed - predicting $499.99
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $519.99
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value via Ensemble Agent
INFO:root:[Ensemble Agent] Running Ensemble Agent - preprocessing text
14:14:30 - LiteLLM:INFO: utils.py:3421 - 
LiteLLM completion() model= openai/gpt-oss-20b; provider = groq
INFO:LiteLLM:
LiteLLM completion() model= openai/gpt-oss-20b; provider = groq
14:14:30 - LiteLLM:INFO: utils.py:1302 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Ensemble Agent] Pre-processed text using groq/openai/gpt-oss-20b
I

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent has found similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent is about to call gpt-5.2 with context including 10 similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent completed - predicting $259.00
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $255.10
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is estimating value via Ensemble Agent
INFO:root:[Ensemble Agent] Running Ensemble Agent - preprocessing text
14:14:43 - LiteLLM:INFO: utils.py:3421 - 
LiteLLM completion() model= openai/gpt-oss-20b; provider = groq
INFO:LiteLLM:
LiteLLM completion() model= openai/gpt-oss-20b; provider = groq
14:14:44 - LiteLLM:INFO: utils.py:1302 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Ensemble Agent] Pre-processed text using groq/openai/gpt-oss-20b
I

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent has found similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent is about to call gpt-5.2 with context including 10 similar products
INFO:root:[OpenAI GPT 5.2 Frontier Agent] OpenAI GPT 5.2 Frontier Agent completed - predicting $189.99
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $181.99
INFO:root:[Autonomous Planning Agent] Autonomous Planning agent is notifying user
INFO:root:[Messaging Agent] Messaging Agent is using GPT-5-MINI to craft the message
14:14:47 - LiteLLM:INFO: utils.py:3421 - 
LiteLLM completion() model= gpt-5-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-5-mini; provider = openai
14:14:54 - LiteLLM:INFO: utils.py:1302 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Messaging Agent] Messaging Agent is sending a push notification
INFO:root:[Messaging 

Opportunity(deal=Deal(product_description='TCL QM5K Series 50QM5K is a 50-inch 4K HDR QD-Mini LED UHD smart TV offering quantum dot color enhancement, Mini-LED backlighting for improved contrast and brightness, smart TV platform features, and support for modern HDR formats.', price=299.99, url='https://www.dealnews.com/TCL-Mini-LED-TV-Super-Bowl-Deals-at-Best-Buy-Up-to-2-000-off-free-shipping/21803677.html?iref=rss-c142'), estimate=519.991, discount=220.00099999999998)